In [2]:
!pip install -q langchain_experimental langchain_core google-generativeai==0.3.1 google-ai-generativelanguage==0.4.0 langchain-google-genai
!pip install -q "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [34]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb==0.4.3 lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.0 MB/s eta 0:00:00


In [35]:
import os

os.environ['GOOGLE_API_KEY']=""

In [36]:
import os
import requests
import zipfile
from io import BytesIO
import textwrap

def download(url,target_folder):

  if not os.path.exists(target_folder):
    os.makedirs(target_folder)

  response = requests.get(url)
  if response.status_code!=200:
    raise Exception(f"Failed to download file :{url}")

  with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
    zip_ref.extractall(target_folder)

  print(f"File extracted to {target_folder}")

In [37]:
def zip(folder_path,zip_file_path):
  with zipfile.ZipFile(zip_file_path,'w',zipfile.ZIP_DEFLATED) as zipf:

    for root, dirs, files in os.walk(folder_path):
      for file in files:
         relative_path = os.path.relpath(os.path.join(root, file), os.path.dirname(folder_path))
         zipf.write(os.path.join(root,file), arcname=relative_path)

  print(f"{zip_file_path} created Successfully")

In [38]:
def wrap_text(text, width=90):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

url = "https://www.dropbox.com/scl/fi/av3nw07o5mo29cjokyp41/singapore_text_files_languages.zip?rlkey=xqdy5f1modtbnrzzga9024jyw&dl=1" # Ensure dl=1 for direct download

folder = "singapore_text"

download(url, folder)

File extracted to singapore_text


In [40]:
!pip show langchain langchain-core

Name: langchain
Version: 0.1.9
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental
---
Name: langchain-core
Version: 0.1.27
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, jsonpatch, langsmith, packaging, pydantic, PyYAML, requests, tenacity
Required-by: langchain, langchain-community, langchain-experimental, langchain-google-genai


In [41]:
import os
import google.generativeai as genai

genai.configure(api_key="")

In [42]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [43]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [44]:
!pip install -q unstructured

In [45]:
%%time
loader = DirectoryLoader('/content/singapore_text/Textfiles3/English/', glob="*.txt", show_progress=False)
docs = loader.load()

CPU times: user 30.9 s, sys: 623 ms, total: 31.5 s
Wall time: 31.6 s


In [46]:
len(docs)


646

In [47]:
docs[0]

Document(page_content='Link: https://www.hellosingaporetours.com/food\n\ntour\n\nTitle: Hello! Singapore Food Tour — Hello! Singapore Tours\n\nAsk any Singaporean what their favourite past time is and they will say eating. Eat like a local on a private food tour and discover why Singaporeans love their food so much!\n\nOur Singapore food tours last 5.5 hours and cover four districts of Singapore; the East Coast (also known as Katong), Old Airport Road, Kampong Glam and Little India. In each district you will enjoy foods from each of the main ethnic groups of Singapore; Chinese, Peranakan, Malay (Muslim) and Indian. There are 10+ food tastings on this tour.\n\nPlease note that there is a wide variety of foods on this tour. If you are unable to consume any of the following then it does mean that you will NOT be able to eat/drink everything on the tour: milk, meat, pork, prawns, fish, wheat/gluten, spice. If you are unable to consume several of those foods then you may prefer to opt for a

In [48]:
raw_text = ""

for i,doc in enumerate(docs):
  text = doc.page_content
  if text:
    raw_text += text

In [49]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex = False
)

In [50]:
texts = text_splitter.split_text(raw_text)

In [51]:
len(texts)

2687

In [52]:
texts[1]

'Please note that there is a wide variety of foods on this tour. If you are unable to consume any of the following then it does mean that you will NOT be able to eat/drink everything on the tour: milk, meat, pork, prawns, fish, wheat/gluten, spice. If you are unable to consume several of those foods then you may prefer to opt for a private customised tour rather than a food tour.\n\nWe offer both a public group and private food tours. Public group tours run on Tuesday, Thursday and Saturday mornings. Private food tours are available 7 days a week and can start any time before 3 pm.Link: https://www.visitsingapore.com/editorials/unique\n\neats\n\nsix\n\nunusual\n\ndining\n\nexperiences\n\nin\n\nsingapore/\n\nTitle: Unique dining experiences in SG\n\nSingapore’s reputation as a food paradise is well warranted — besides delicious hawker food and world-class Michelin fare, The Lion City is also home to a range of experiential and immersive culinary experiences.'

In [54]:
!pip install -q hnswlib==0.7.0 chroma-hnswlib==0.7.1 chromadb==0.4.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [61]:
!pip uninstall -q hnswlib

Proceed (Y/n)? y


In [72]:
!pip install -q faiss-gpu faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.7 MB/s eta 0:00:00


In [73]:
%%time
from langchain.vectorstores.faiss import FAISS

vectorstore = FAISS.from_texts(texts,embeddings)

vectorstore.save_local("index")



CPU times: user 9.98 s, sys: 147 ms, total: 10.1 s
Wall time: 54.8 s


In [74]:
vectorstore = FAISS.load_local("index",embeddings)

In [76]:
query = "Tell me about Universal Studios Singapore?"

vectorstore.similarity_search(query, k=5)

[Document(page_content="You may also want to check out the other interactive activities to get a taste of city planning and insights into the country's conservation efforts. It's a multi-sensory experience that takes you deep into Singapore's exciting planning journey.Link: https://www.visitsingapore.com/see\n\ndo\n\nsingapore/recreation\n\nleisure/fun\n\nthings\n\nto\n\ndo/universal\n\nstudios\n\nsingapore/\n\nTitle: Universal Studios Singapore\n\nThe shimmering wonders of the silver screen comes to vivid life at Universal Studios Singapore, the first-ever Hollywood movie theme park in Southeast Asia.\n\nWhether you’re a daredevil in search of adrenaline rushes and high-flying thrills, a film buff looking to immerse yourself in the world of movies, or in search of a thrilling jaunt with your loved ones, this theme park will provide you with a ride to remember.\n\nFun for the Family"),
 Document(page_content='on\n\ntravelling\n\nwith\n\nkids/\n\nTitle: Tips on travelling with kids to S

In [77]:
retriever = vectorstore.as_retriever(k=5) # can add mmr fetch_k=20, search_type="mmr"

retriever.get_relevant_documents(query)[1]

Document(page_content='on\n\ntravelling\n\nwith\n\nkids/\n\nTitle: Tips on travelling with kids to Singapore\n\nFamily holidays are the best for getting quality time with the whole brood, but they can be as frazzling as they are fun-filled, especially with tiny tots in tow. Fret not, parents, we’ve got you—here’s a list of the most family-friendly experiences in Singapore.\n\nCheck in to family\n\nfriendly hotels')

In [78]:
from operator import itemgetter

from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [80]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.8)

In [81]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [82]:
text_reply = chain.invoke("Tell me about Universal Studio Singapore")

print(wrap_text(text_reply))

Universal Studios Singapore is the first-ever Hollywood movie theme park in Southeast
Asia. It offers thrilling rides, live shows, and immersive themed zones inspired by your
favorite movies. You can buy Universal Studios Singapore tickets and enjoy an unbeatable
discount of 40% from MySingaporePass. The pass provides a hassle-free booking process,
allowing you to skip the long queues and gain easy access to the park.


# **With RagFusion**

In [84]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

In [85]:
prompt = ChatPromptTemplate(
    input_variables=['original_query'],
    messages=[
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))
        ]
    )

In [86]:
prompt

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generates multiple search queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to: {question} \n OUTPUT (4 queries):'))])

In [100]:
generate_queries = (
    prompt
    | ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.8,convert_system_message_to_human=True)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [101]:
original_query = "universal studios Singapore"

In [102]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [103]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [104]:
!pip install -q langchain

In [105]:
import langchain
langchain.debug = True

In [106]:
ragfusion_chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [107]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "universal studios Singapore"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "universal studios Singapore"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful assistant that generates multiple search queries based on a single input query.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          

[(Document(page_content='Marina Bay Sands—Luxury retail paradise'),
  0.04972677595628415),
 (Document(page_content='Where To Stay In Singapore\n\nWhether you are looking for that once-in-a-lifetime luxury hotel, a small intimate boutique hotel, a value-for-money mid-range hotel, or a budget guesthouse, Singapore has a comprehensive collection of them all.\n\nYou must be warned, though, that Singapore is among the most expensive cities in Asia. Singapore has been dubbed the world’s most expensive city to live in for the fifth year in a row, which is also reflected in the hotel prices. That said, in this guide, we reveal some good value options for your money, so there is no need to panic. 🙂'),
  0.048924731182795694),
 (Document(page_content='Daily 8.30am\n\n6pm.\n\nNight Safari—Hang out at Singapore’s no.1 night spot'),
  0.04841269841269841),
 (Document(page_content="You may also want to check out the other interactive activities to get a taste of city planning and insights into the 

In [108]:
from langchain.schema.runnable import RunnablePassthrough
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

In [110]:
full_rag_fusion_chain.input_schema.schema()

{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question', 'type': 'string'}}}

In [111]:
full_rag_fusion_chain.invoke({"question": "Tell me about Universal Studio Singapore?"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Tell me about Universal Studio Singapore?"
}

[chain/end] [1:chain:RunnableSequen

'Universal Studios Singapore is the first-ever Hollywood movie theme park in Southeast Asia. It offers thrilling rides and entertainment for all ages, including adrenaline-pumping roller coasters, live shows, and immersive themed zones inspired by your favorite movies.'